<a href="https://colab.research.google.com/github/lava-chen/ML_study/blob/main/%E7%94%A824%E4%B8%AA%E6%B5%8B%E8%AF%95%E5%87%BD%E6%95%B0%E6%B5%8B%E8%AF%95PSO%E7%AE%97%E6%B3%95%E6%80%A7%E8%83%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **用24个测试函数测试PSO算法性能**
> P:Particle S:swarm O:optimization

In [19]:
# @title 1.加载依赖项代码
import os
import numpy as np

## 2.定义24个测试函数

In [20]:
# @title 2.1.Unimodal单峰函数

def F1(x):
    return np.sum(x**2)
def F2(x):
    return np.sum(np.abs(x)) + np.prod(np.abs(x))
def F3(x):
    return np.sum([np.sum(x[:i+1])**2 for i in range(len(x))])
def F4(x):
    return np.max(np.abs(x))
def F5(x):
    return np.sum(100 * (x[1:] - x[:-1]**2)**2 + (x[:-1] - 1)**2)
def F6(x):
    return np.sum(np.floor(x + 0.5)**2)
def F7(x):
    return np.sum([i * (xi**4) for i, xi in enumerate(x, 1)]) + np.random.uniform(0, 1)
def F24(x):
    return np.sum(np.abs(x) * (np.arange(len(x)) + 1))

In [21]:
# @title 2.2.Multimodal多峰函数
def F8(x):
    return np.sum(-x * np.sin(np.sqrt(np.abs(x))))
def F9(x):
    return np.sum(x**2 - 10 * np.cos(2 * np.pi * x) + 10)
def F10(x):
    return -20 * np.exp(-0.2 * np.sqrt(np.mean(x**2))) - np.exp(np.mean(np.cos(2 * np.pi * x))) + 20 + np.e
def F11(x):
    return np.sum(x**2) / 4000 - np.prod(np.cos(x / np.sqrt(np.arange(1, len(x) + 1)))) + 1
def F12(x):
    y = 1 + (x + 1) / 4
    return np.pi / len(x) * (10 * np.sin(np.pi * y[0])**2 + np.sum((y[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * y[1:])**2)) + (y[-1] - 1)**2) + np.sum([penalty(xi) for xi in x])
def penalty(xi, a=10, k=100, m=4):
    if xi > a:
        return k * (xi - a)**m
    elif xi < -a:
        return k * (-xi - a)**m
    return 0
def F13(x):
    return 0.1 * (np.sin(3 * np.pi * x[0])**2 + np.sum((x[:-1] - 1)**2 * (1 + np.sin(3 * np.pi * x[1:])**2)) + (x[-1] - 1)**2 * (1 + np.sin(2 * np.pi * x[-1])**2)) + np.sum([penalty(xi, 5, 100, 4) for xi in x])


In [22]:
# @title 2.3.Fixed固定维度的多峰函数
def F14(x):
    a = np.array([[3, 5, 2, 1, 7], [5, 2, 1, 4, 9]])
    return 1 / (0.002 + np.sum(1 / (np.arange(1, len(x)+1) + np.sum((x - a[:, :len(x)])**6, axis=1))))
def F15(x):
    a = np.array([1.0, 1.5, 2.0])
    return np.sum([(a[i] - (x[0]*(x[1]**2 + x[2])))**2 for i in range(len(a))])
def F16(x):
    return 4 * x[0]**2 - 2.1 * x[0]**4 + x[0]**6 / 3 + x[0] * x[1] + 4 * x[1]**2 * (-4 + 4 * x[1]**2)
def F17(x):
    return (x[1] - (5.1 / (4 * np.pi**2)) * x[0]**2 + 5 * x[0] / np.pi - 6)**2 + 10 * (1 - 1 / (8 * np.pi)) * np.cos(x[0]) + 10
def F18(x):
    return (1 + (x[0] + x[1] + 1)**2 * (19 - 14 * x[0] + 3 * x[0]**2 - 14 * x[1] + 6 * x[0] * x[1] + 3 * x[1]**2)) * (30 + (2 * x[0] - 3 * x[1])**2 * (18 - 32 * x[0] + 12 * x[0]**2 + 48 * x[1] - 36 * x[0] * x[1] + 27 * x[1]**2))
def F19(x):
    return -np.sum(np.array([1 / (x[0] - a)**2 + 1 / (x[1] - b)**2 for a, b in zip(np.random.rand(10), np.random.rand(10))]))
def F20(x):
    return -np.sum(np.array([1 / (x[0] - a)**2 + 1 / (x[1] - b)**2 for a, b in zip(np.random.rand(10), np.random.rand(10))]))

def F21(x):
    return -np.sum(np.array([1 / (x[0] - a)**2 + 1 / (x[1] - b)**2 for a, b in zip(np.random.rand(10), np.random.rand(10))]))

def F22(x):
    return -np.sum(np.array([1 / (x[0] - a)**2 + 1 / (x[1] - b)**2 for a, b in zip(np.random.rand(10), np.random.rand(10))]))

def F23(x):
    return -np.sum(np.array([1 / (x[0] - a)**2 + 1 / (x[1] - b)**2 for a, b in zip(np.random.rand(10), np.random.rand(10))]))

## 3.PSO算法原理及代码实现部分
### 3.1.简介
- 粒子群优化算法(PSO)是一种**进化算法**(evolutionary computation)
- 1995 年由Eberhart 博士和kennedy 博士提出，源于对鸟群捕食的行为研究 。

### 3.2.原理的通俗理解
*设想这样一个场景：一群鸟在随机搜索食物。在这个区域里只有一块食物。所有的鸟都不知道食物在那里。但是他们知道当前的位置离食物还有多远。那么找到食物的最优策略是什么呢。最简单有效的就是搜寻目前离食物最近的鸟的周围区域。*

*鸟群在整个搜寻的过程中，通过相互传递各自的信息，让其他的鸟知道自己的位置，通过这样的协作，来判断自己找到的是不是最优解，同时也将最优解的信息传递给整个鸟群，最终，整个鸟群都能聚集在食物源周围，即找到了最优解。*

### 3.3.PSO流程

#### 3.3.1关于速度与位置
将鸟简化成粒子，他的位置表示成$$X=(x_1,x_2,x_3,...,x_n)$$他的速度表示成$$V=(v_1,v_2,v_3,...,v_n)$$
每一个粒子都知道:
1. 目前为止**自己**发现的最好的位置**`pbest`**
2. 目前为止**所有粒子**发现的最好位置**`gbest`**
3. 现在的位置 $X_i$

#### 3.3.2.速度和位置的更新
1. 速度的更新
  $$V_{i(t+1)}=wV_{i(t)}+c_1r(pbest_i-X_i)+c_2r(gbest_i-X_i)$$
2. 位置的更新
  $$X_{i(t+1)}=X_{i(t)}+V_{i(t+1)}$$

|符号        ..|含义.           |
|-----------|------------------|
|$w$        |惯性权重           |
|$r_1,r_2$  |介于(0,1)之间的随机数|
|$c_1,c_2$  |学习因子,通常等于2  |

#### 3.3.3.关于w
值为非负，值越大，全局搜索能力越强，局部搜索能力弱，反之相反。


In [23]:
# @title PSO参数
w = 0.7  # 惯性权重
c1 = 1.5  # 个体学习因子
c2 = 1.5  # 群体学习因子
num_particles = 50  # 粒子数量
max_iter = 1000  # 最大迭代次数

In [24]:
# @title 初始化粒子群
def init_PSO(dim, bounds):
    # 随机初始化粒子的位置和速度
    positions = np.random.uniform(bounds[0], bounds[1], (num_particles, dim))
    velocities = np.random.uniform(-1, 1, (num_particles, dim))
    return positions, velocities

In [25]:
# @title PSO核心算法
def PSO(func, dim, bounds):
    '''dim:纬度;bounds:范围'''
    positions, velocities = init_PSO(dim, bounds)
    pbest_positions = np.copy(positions)
    pbest_scores = np.array([func(p) for p in positions])
    gbest_position = positions[np.argmin(pbest_scores)]
    gbest_score = np.min(pbest_scores)

    for t in range(max_iter):
        for i in range(num_particles):
            r1 = np.random.rand(dim)
            r2 = np.random.rand(dim)
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest_positions[i] - positions[i]) +
                             c2 * r2 * (gbest_position - positions[i]))
            positions[i] += velocities[i]
            positions[i] = np.clip(positions[i], bounds[0], bounds[1])

            score = func(positions[i])
            if score < pbest_scores[i]:
                pbest_scores[i] = score
                pbest_positions[i] = positions[i]

        # 更新全局最佳
        if np.min(pbest_scores) < gbest_score:
            gbest_score = np.min(pbest_scores)
            gbest_position = pbest_positions[np.argmin(pbest_scores)]

    return gbest_position, gbest_score


In [26]:
# @title 测试函数的配置
functions = [
    (F1, 30, [-100, 100]),
    (F2, 30, [-10, 10]),
    (F3, 30, [-100, 100]),
    (F4, 30, [-100, 100]),
    (F5, 30, [-30, 30]),
    (F6, 30, [-100, 100]),
    (F7, 30, [-1.28, 1.28]),
    (F8, 30, [-500, 500]),
    (F9, 30, [-5.12, 5.12]),
    (F10, 30, [-32, 32]),
    (F11, 30, [-600, 600]),
    (F12, 30, [-50, 50]),
    (F13, 30, [-50, 50]),
    (F14, 2, [-65.536, 65.536]),
    (F15, 4, [-5, 5]),
    (F16, 2, [-5, 5]),
    (F17, 2, [-5, 10]),
    (F18, 2, [-2, 2]),
    (F19, 3, [0, 1]),
    (F20, 6, [0, 1]),
    (F21, 4, [0, 10]),
    (F22, 4, [0, 10]),
    (F23, 4, [0, 10]),
    (F24, 30, [-100, 100])
]

## 4.求解

In [27]:
for func, dim, bounds in functions:
    best_position, best_score = PSO(func, dim, bounds)
    print(str(func),f"--最佳解: {best_score}")

<function F1 at 0x7e1ed6aa4310> --最佳解: 2.772765917292112e-21
<function F2 at 0x7e1ed6aa4280> --最佳解: 10.000008095271244
<function F3 at 0x7e1ed6aa4af0> --最佳解: 5001.695756553935
<function F4 at 0x7e1ed6aa4670> --最佳解: 0.5017768637405013
<function F5 at 0x7e1ed6aa4ca0> --最佳解: 26.49970777746277
<function F6 at 0x7e1ed6aa4ee0> --最佳解: 4.0
<function F7 at 0x7e1ed6aa5f30> --最佳解: 0.016597601099969408
<function F8 at 0x7e1ed6aa44c0> --最佳解: -8077.472539735808
<function F9 at 0x7e1ed6aa6560> --最佳解: 98.5715337302592
<function F10 at 0x7e1ed6aa57e0> --最佳解: 2.95769950714303
<function F11 at 0x7e1ed6aa5c60> --最佳解: 0.04658538693495773
<function F12 at 0x7e1ed6aa5fc0> --最佳解: 2.8061600772161777
<function F13 at 0x7e1ed6aa7490> --最佳解: 0.010987365835884694
<function F14 at 0x7e1ed6aa5900> --最佳解: 0.9961489071397479
<function F15 at 0x7e1ed6aa56c0> --最佳解: 0.5
<function F16 at 0x7e1ed6aa79a0> --最佳解: -4.031442229302004
<function F17 at 0x7e1ed6aa53f0> --最佳解: 0.39788735772973816
<function F18 at 0x7e1ed6aa5bd0> 